# Parse the FIX message of a trade order 

More exactly, of a `Order Mass Status Request` (`35=AF`).

The replies are called `Execution Report` and they start with `35=8`.

There are several, parsed with a while loop, from which we want to return an updated list of positions.

## Example

`8=FIX.4.4|9=230|35=8|34=2402|49=cServer|50=TRADE|52=20230611-12:04:08.569|56=demo.icmarkets.8739125|57=Rmp1iP7xn|11=17|14=0|37=543125891|38=1|39=0|40=2|44=34100|54=2|55=10015|59=1|60=20230610-17:06:19.069|150=I|151=1|584=64|721=339826567|911=110|10=226|`

## Relevant Fields

`55=14|710=63|721=339825679|727=10|728=0|730=1.59431|702=1|704=1000|705=0|10=215|`

* 37 = OrderID (order_id, or the order ID as recorded by cTrader)
* 11 = ClOrdID (unique identified for the order, allocated by the client - could be used to assign different sources of orders, like different strategies, if they both go to the same broker)
* 911 = TotNumReports (total number of opened orders, that are returned now)
* 150 = ExecType (0=New, 4=Canceled, 5=Replace, 6=Rejected; C=Expired; F=Trade; I=Order Status)
* 39 = OrdStatus (0=New, 1=Partially Filled, 2=Filled, 3=Rejected, 4=Canceled)
* 55 = Symbol (asset name)
* 54 = Side (1=Buy, 2=Sell)
* 60 = TransactTime (time of this execution report in UTC)
* 6 = AvgPx (the average price at which the deal was filled, for IOC or GTD it is the VWAP - volume weighted average price) of the filled order
* 38 = OrderQty (number of units bought, it can also be fractional for BTC)
* 151 = LeavesQty (amount of units still to be filled. The value is between 0 and OrderQty when partially filled)
* 14 = CumQty (total amount of order which has been filled)
* 32 = LastQty (bought or sold quantity of the order that has been filled in this last fill)
* 40 = OrdType (1=Market, 2=Limit) - No Stop?
* 44 = Limit Price if given in NewOrderSingle
* 99 = Stop Price if given in NewOrderSingle
* 59 = TimeInForce (1=GTC - Good Till Cancel, 3= IOC - Immediate or Cancel, 6=GTD - Good Till Date)
* 126 = ExpireTime (Timestamp if privided in NewOrderSingle)
* 58 = Some text with explanations
* 103 = OrdRejReason, as a number integer
* 721 = PosMaintRptID = the position ID to which the order is attached
* 494 = Designantion = a string client custom order label
* 584 = MassStatusReqID = the unique ID of mass status request as assigned by the client

These below not seen in our messages though:

* 1000 = Absolute TP
* 1001 = Relative TP
* 1002 = Absolute SL
* 1003 = Relative SL
* 1004 = Trailing SL
* 1005 = Trigger Method SL
* 1006 = Guaranteed SL

In [ ]:
import re

from assets import assets_all, DICT_SYMBOL_ID_SYMBOL

In [ ]:
DICT_SYMBOL_ID_SYMBOL

In [ ]:
messages = [
    "8=FIX.4.4|9=230|35=8|34=3607|49=cServer|50=TRADE|52=20230611-12:03:03.432|56=demo.icmarkets.8739125|57=Fr5w3K1xj|11=17|14=0|37=543125891|38=1|39=0|40=2|44=34100|54=2|55=10015|59=1|60=20230610-17:06:19.069|150=I|151=1|584=94|721=339826567|911=110|10=155|"
]
messages

In [ ]:
def parse_one_order_message(full_message):
    d = None
    if match := re.search("37=(\\d+)", full_message):
        d = {}
        d["order_id"] = match.group(1)
        #
        if match := re.search("11=(\\d+)", full_message):
            order_request_id = match.group(1)
        else:
            order_request_id = None
        #
        if match := re.search("721=(\\d+)", full_message):
            position_id = match.group(1)
        else:
            position_id = None
        #
        if match := re.search("55=(\\d+)", full_message):
            symbol_id = int(match.group(1))
        else:
            symbol_id = 0
        #
        if match := re.search("38=([\d.]+)\|", full_message):
            quantity_ordered = float(match.group(1))
        else:
            quantity_ordered = 0.0
        #
        if match := re.search("14=([\d.]+)\|", full_message):
            quantity_filled = float(match.group(1))
        else:
            quantity_filled = 0.0
        #
        if match := re.search("151=([\d.]+)\|", full_message):
            quantity_not_filled = float(match.group(1))
        else:
            quantity_not_filled = 0.0
        #
        if match := re.search("39=(\\d+)", full_message):
            order_status = match.group(1)
        else:
            order_status = None
        #
        if match := re.search("54=(\\d+)", full_message):
            value = match.group(1)
            order_direction = "buy" if value == "1" else "sell" if value == "2" else None  # noqa
        else:
            order_direction = None
        #
        if match := re.search("40=(\\d+)", full_message):
            value = match.group(1)
            order_type = "market" if value=="1" else "limit" if value=="2" else "stop" if value=="3" else None
        else:
            order_type = None
        #
        if match := re.search("44=([\d.]+)\|", full_message):
            price_limit = float(match.group(1))
        else:
            price_limit = None
        #
        if match := re.search("99=([\d.]+)\|", full_message):
            price_stop = float(match.group(1))
        else:
            price_stop = None
        #
        if match := re.search("59=(\\d+)", full_message):
            value = match.group(1)
            time_in_force = "GTC" if value=="1" else "IOC" if value=="3" else "GTD" if value==6 else None
        else:
            symbol_id = 0
        #
        # this to allow any character, numbers or letters from 150= to the first |
        if match := re.search(r"150=([^|]+)", full_message):
            value = match.group(1)
            execution_type = "new" if value=="0" else "canceled" if value=="4" else "replaced" if value=="5" else "rejected" if value=="8" else "expired" if value=="C" else "trade" if value=="F" else "order_status" if value=="I" else None
        else:
            execution_type = None
        #
        if match := re.search(r"60=([^|]+)", full_message):
            datetime = match.group(1)
        else:
            datetime = None
        #
        if match := re.search("911=(\\d+)", full_message):
            num_opened_orders = int(match.group(1))
        else:
            num_opened_orders = 0
        
            
        # build dictionary
        d["position_id"] = position_id
        d["datetime"] = datetime
        d["symbol"] = DICT_SYMBOL_ID_SYMBOL[symbol_id]
        d["symbol_id"] = symbol_id
        d["order_direction"] = order_direction
        d["order_type"] = order_type
        d["price_limit"] = price_limit
        d["price_stop"] = price_stop
        d["quantity_ordered"] = quantity_ordered
        d["quantity_filled"] = quantity_filled
        d["quantity_not_filled"] = quantity_not_filled
        d["order_status"] = order_status
        d["time_in_force"] = time_in_force
        d["execution_type"] = execution_type
        d["order_request_id"] = order_request_id
        d["num_opened_orders"] = num_opened_orders
        
    print(f"d={d}")
    return d

orders = []
try:
    for message in messages:
        d = parse_one_order_message(message)
        if d is not None:
            orders.append(d)
except:
    print(f"TRADE Unable to read from server")

# for order in orders:
#    print(f"order={order}")
orders